## 5 folds cross-validation: 60/% train - 20/% validation - 20%test

In [3]:
import os
import numpy as np
import shutil
from sklearn.model_selection import StratifiedKFold, train_test_split

def prepare_and_save_folds(train_dir, test_dir, folds_dir, id, n_splits_outer=5):

    if os.path.exists(folds_dir):
        shutil.rmtree(folds_dir)
        print(f"Carpeta '{folds_dir}' eliminada.")
    os.makedirs(folds_dir)
    print(f"Carpeta '{folds_dir}' creada.")

    train_files = [f for f in os.listdir(train_dir) if f.endswith('_X.npy')]
    labels = [0 if 'AVPEPUDEAC' in f else 1 for f in train_files]

    test_files = [f for f in os.listdir(test_dir) if f.endswith('_X.npy')]
    test_labels = [0 if 'AVPEPUDEAC' in f else 1 for f in test_files]
    print(train_files==test_files)

    skf_outer = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

    for fold_idx, (train_val_index, test_index) in enumerate(skf_outer.split(train_files, labels)):
        # Print the test files for the current fold
        print(f"Fold {fold_idx + 1} Test Files:")
        for i in test_index:
            print(f"File: {test_files[i]}, Label: {test_labels[i]}")

        # Load test data
        test_data = [np.load(os.path.join(test_dir, test_files[i])) for i in test_index]
        test_data = np.vstack(test_data)
        test_labels_fold = np.array([test_labels[i] for i in test_index])
        test_labels_fold = np.repeat(test_labels_fold, test_data.shape[0] // len(test_labels_fold))
        test_data, test_labels_fold = shuffle_data(test_data, test_labels_fold)  # Shuffle test data


        # Assign data to train and validation using the inner split
        train_index, val_index = train_test_split(train_val_index, test_size=20, stratify=[labels[i] for i in train_val_index], shuffle=True, random_state=42)

        # Print the train files for the current fold
        print(f"Fold {fold_idx + 1} Train Files:")
        for i in train_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        # Print the validation files for the current fold
        print(f"Fold {fold_idx + 1} Validation Files:")
        for i in val_index:
            print(f"File: {train_files[i]}, Label: {labels[i]}")

        train_data = [np.load(os.path.join(train_dir, train_files[i])) for i in train_index]
        train_data = np.vstack(train_data)
        train_labels = np.array([labels[i] for i in train_index])
        train_labels = np.repeat(train_labels, train_data.shape[0] // len(train_labels))
        train_data, train_labels = shuffle_data(train_data, train_labels)  # Shuffle train data

        val_data = [np.load(os.path.join(train_dir, train_files[i])) for i in val_index]
        val_data = np.vstack(val_data)
        val_labels = np.array([labels[i] for i in val_index])
        val_labels = np.repeat(val_labels, val_data.shape[0] // len(val_labels))
        val_data, val_labels = shuffle_data(val_data, val_labels)


        # Save the data to .npy files
        np.save(os.path.join(folds_dir, f'X_train_{id}_fold_{fold_idx+1}.npy'), train_data)
        np.save(os.path.join(folds_dir, f'y_train_{id}_fold_{fold_idx+1}.npy'), train_labels)
        np.save(os.path.join(folds_dir, f'X_val_{id}_fold_{fold_idx+1}.npy'), val_data)
        np.save(os.path.join(folds_dir, f'y_val_{id}_fold_{fold_idx+1}.npy'), val_labels)
        np.save(os.path.join(folds_dir, f'X_test_{id}_fold_{fold_idx+1}.npy'), test_data)
        np.save(os.path.join(folds_dir, f'y_test_{id}_fold_{fold_idx+1}.npy'), test_labels_fold)

        print(train_index, val_index, test_index)

    print("Archivos .npy guardados para cada fold en la carpeta de folds.")

def shuffle_data(data, labels):
    shuffle_indices = np.random.permutation(len(labels))
    data = data[shuffle_indices]
    labels = labels[shuffle_indices]
    return data, labels

In [4]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_N',
    id='real_N'
)

Carpeta '/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_N' eliminada.
Carpeta '/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_N' creada.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_N'

In [ ]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/generated_together_train_40_1e5_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_40_1e5_N',
    id='40_1e5_N'
)

In [ ]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/generated_together_train_40_1e5_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/generated_together_train_40_1e5_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_40_1e5_N',
    id='40_1e5_N'
)

In [ ]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/generated_together_train_40_1e5_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_10ms_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_10ms_N',
    id='40_1e5_N'
)

In [ ]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_10ms_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_10ms_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_all_10ms_N',
    id='40_1e5_N'
)

In [ ]:
prepare_and_save_folds(
    train_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/generated_together_train_40_1e5_N',
    test_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/test_together_syn_1_N',
    folds_dir='/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_1_N',
    id='40_1e5_N'
)

In [ ]:
a = np.load('/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_40_1e5_N/X_test_40_1e5_N_fold_2.npy')
b = np.load('/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_40_1e5_N/X_test_40_1e5_N_fold_2.npy')
c = np.load('/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_10ms_N/X_test_40_1e5_N_fold_2.npy')
d = np.load('/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_syn_1_N/X_test_40_1e5_N_fold_2.npy')
e = np.load('/Users/carlos/Downloads/Neuroevolution/data/sets/folds_5/files_real_N/X_test_real_N_fold_2.npy')
